In [1]:
# # old logging
# from applicationinsights import TelemetryClient  
# from applicationinsights.logging import enable
# import os
# from dotenv import load_dotenv
# load_dotenv("credentials.env")

# instrumentation_key = os.environ.get("APPLICATION_INSIGHTS_INSTRUMENTATION_KEY"  )
# client = TelemetryClient(instrumentation_key)  
# enable(instrumentation_key)  

# event_properties = {  
#     'property1': 'value1',  
#     'property2': 'value2'  
# }  
# client.track_event('LanguageModelResult', event_properties)  
# client.flush()  


In [2]:
import os
import random
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import ConversationalChatAgent, AgentExecutor, Tool
from langchain.memory import CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager

from azure.monitor.events.extension import track_event
from azure.monitor.opentelemetry import configure_azure_monitor

#custom libraries that we will use later in the app
from common.utils_povel import DocSearchTool, CSVTabularTool, SQLDbTool, ChatGPTTool, BingSearchTool, run_agent, get_search_results
from common.callbacks import StdOutCallbackHandler
from common.prompts_povel import CUSTOM_CHATBOT_PREFIX, CUSTOM_CHATBOT_SUFFIX 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

MODEL_DEPLOYMENT_NAME = "gpt-4-32k" # Reminder: gpt-35-turbo models will create parsing errors and won't follow instructions correctly 
# MODEL_DEPLOYMENT_NAME = "gpt-35-turbo-16k" # Reminder: gpt-35-turbo models will create parsing errors and won't follow instructions correctly

In [3]:
configure_azure_monitor(disable_metric=True, connection_string=os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING"))

In [4]:
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [5]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.2, max_tokens=800)

# Uncomment the below line if you want to see the responses being streamed/typed
# llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=500, streaming=True, callback_manager=cb_manager)

In [6]:
# vector_only_indexes = ["digge-ekonomi-index-files-on-your-data"] # funkar bättre på fråga : "@docsearch, Vilket konto ska jag fakturera egenavgift för hjälpmedel?"
vector_only_indexes = ["1b-digge-100-full", "1b-digge-ekonomi-full"]

doc_search = DocSearchTool(
    llm=llm,
    vector_only_indexes=vector_only_indexes,
    k=10,
    similarity_k=5,
    reranker_th=1,
    sas_token=os.environ["BLOB_SAS_TOKEN"],
    callback_manager=cb_manager,
    return_direct=True,
    # This is how you can edit the default values of name and description
    name="@docsearch",
    description="useful when the questions includes the term: @docsearch.\n",
)

# SQLDbTool is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLDbTool(llm=llm, k=30, callback_manager=cb_manager, return_direct=True)

# ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager, return_direct=True)

# BingSearchTool is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchTool(llm=llm, k=5, callback_manager=cb_manager, return_direct=True)

In [7]:
tools = [www_search, sql_search, doc_search, chatgpt_search]

In [8]:
cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="Agent-Test-Session" + str(random.randint(1, 1000)),
    user_id="Agent-Test-User" + str(random.randint(1, 1000))
    )
# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [9]:
llm_a = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.2, max_tokens=500)
agent = ConversationalChatAgent.from_llm_and_tools(llm=llm_a, tools=tools, system_message=CUSTOM_CHATBOT_PREFIX, human_message=CUSTOM_CHATBOT_SUFFIX)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=10, chat_memory=cosmos)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, handle_parsing_errors=True)

# # Let's see the custom prompt prefix we created for our brain agent
# printmd(agent_chain.agent.llm_chain.prompt.messages[0].prompt.template)

# # Also let's see the Prompt that the Agent uses to talk to the LLM
# printmd(agent_chain.agent.llm_chain.prompt.messages[2].prompt.template)

In [10]:
QUESTION = "@docsearch, Vilket konto ska jag fakturera egenavgift för hjälpmedel?"
response = run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })


Tool: @docsearch


Du bör fakturera egenavgift för hjälpmedel på konto 3083<sup><a href="https://blobstoragejd5ypzfx2l6vi.blob.core.windows.net/digge-ekonomi-dokument/Artikelf%C3%B6rteckning%20f%C3%B6r%20Agresso%20weborder.pdf" target="_blank">[1]</a></sup>.

In [12]:
QUESTION = "@docsearch, Hur fungerar ingående moms?"
response = run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })

Tool: @docsearch


Ingående moms fungerar på följande sätt: När man köper varor och tjänster till verksamheten kan man dra av den ingående momsen. Om kostnaden för förtäring per person överstiger 300 kr exklusive moms, kan hela den ingående momsen dras av. Om kostnaden överstiger 300 kr exklusive moms per person, begränsas avdraget till den ingående momsen som beräknas på beloppet 300 kr exklusive moms. För andra kostnader, som entréavgifter till konserter eller liknande, får avdrag göras med momsen på ett underlag om 180 kronor exklusive moms. Vid försäljning av varor inom EU kan försäljning utan moms göras om köparen är momsregistrerad i ett annat EU-land och köparens VAT-nummer anges på fakturan. Vid försäljning utanför EU räknas det som export och ingen moms tillkommer. För mer information och exempel, se Momsguiden<sup><a href="https://blobstoragejd5ypzfx2l6vi.blob.core.windows.net/digge-ekonomi-dokument/Momsguiden.pdf" target="_blank">[1]</a></sup> och Kontoplan<sup><a href="https://blobstoragejd5ypzfx2l6vi.blob.core.windows.net/digge-ekonomi-dokument/Kontoplan.pdf" target="_blank">[2]</a></sup>.

In [ ]:
# QUESTION = "@sqlsearch, How many people died of covid in Texas in 2020?"
QUESTION = "@sqlsearch, How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states? Use the hospitalizedIncrease column?"

response=run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })

In [ ]:
QUESTION = "@bing, I need to take my girlfriend to dinner tonight in downtown Chicago. Please give me options for Italian and Sushi as well"
response=run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })

In [ ]:
QUESTION = """
compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

- ADN Registerd Nurse 
- Occupational therapist assistant
- Dental Hygienist
- Graphic Designer
- Real Estate Agent

Create a table with your findings. Place the sources on each cell.
"""
response=run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })

In [ ]:
QUESTION = "@chatgpt, tell me the formula in physics for momentum"
response=run_agent(QUESTION, agent_chain)
printmd(response)

track_event("LanguageModelResult",
            {
                "Prompt": QUESTION, #User input text
                "Completion": response, # AI-generated text
            })